In [ ]:
from energyclustering.data.fluvius import read_data_pickle
import energyclustering.clustering.similarity.matching_similarity as simple_matching
import numpy as np 
import pandas as pd 
from dask.distributed import Client
from pathlib import Path

In [ ]:
%load_ext autoreload
%autoreload 2
PRE_PATH = Path('/cw/dtaiproj/ml/2020-FLAIR-VITO/profile-clustering/distance_matrix_v2')
PRE_PATH.mkdir(parents = True, exist_ok= True)

In [ ]:
info_df, data_df = read_data_pickle()
data_df = data_df.sort_index(ascending = False)
data_df = data_df.groupby('meterID').head(1)
data_df

In [ ]:
data_subset_df = data_df.sample(200, random_state= 123)

In [ ]:
data_subset_df.to_pickle(PRE_PATH/'data.pkl')
info_df.loc[data_subset_df.index].to_pickle(PRE_PATH/'info.pkl')

In [ ]:
%%time
with Client(n_workers = 40, threads_per_worker = 1, local_directory = '/cw/dtailocal/jonass') as client: 
    dist_df = simple_matching.distance_matrix_smart(data_subset_df, client, window = 4, total_blocks = 200)

In [ ]:
distance_df = pd.DataFrame(dist_df, index = data_subset_df.index, columns = data_subset_df.index)
distance_df.to_pickle('full_distance_matrix.pkl')

In [ ]:
blocks = simple_matching.generate_blocks(10, total_blocks=25)
blocks

In [ ]:
40.9/100 *(2200 * 2200)/60/60

In [ ]:
np.array(simple_matching.linear_sum_timings).mean()

In [ ]:
mean_execution_time = (48)/45
mean_execution_time

In [ ]:
nb_of_distances = (data_df.shape[0]*(data_df.shape[0]-1))/2
nb_of_distances

In [ ]:
(nb_of_distances*mean_execution_time)/60/60